<a href="https://colab.research.google.com/github/axel-sirota/model_training_best_practices/blob/master/module3/ModelTraining_Mod3Demo2_CVSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross Validation: Determining performance on a grid


First let's download the dataset

In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt


In [2]:

%%writefile get_data.sh
mkdir -p data
if [ ! -f data/glass.csv ]; then
  wget -O data/glass.csv https://www.dropbox.com/scl/fi/dv522a61am4dsc3vkfp4p/glass.csv?rlkey=6l9v685sw98plzj2myvtjpes6&dl=0
fi

Writing get_data.sh


In [3]:
!bash get_data.sh

--2023-12-08 18:19:23--  https://www.dropbox.com/scl/fi/dv522a61am4dsc3vkfp4p/glass.csv?rlkey=6l9v685sw98plzj2myvtjpes6
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uce1635c1f89abbc4e6f5ff98ac1.dl.dropboxusercontent.com/cd/0/inline/CJD1D19Y6xvWu1AHAtPgrJOdqXSqy-8XSrqjKGPA69Atbzuh55i_erlJmeJKUQpP0_X3Z7BFNFiHowsPZo0FFw9BCEMjUxFEOmlSAGEAAWo0iDbhkYtAWP9jR-OiL3CpLiof0jVDDWEEN67PEeRCnvkr/file# [following]
--2023-12-08 18:19:23--  https://uce1635c1f89abbc4e6f5ff98ac1.dl.dropboxusercontent.com/cd/0/inline/CJD1D19Y6xvWu1AHAtPgrJOdqXSqy-8XSrqjKGPA69Atbzuh55i_erlJmeJKUQpP0_X3Z7BFNFiHowsPZo0FFw9BCEMjUxFEOmlSAGEAAWo0iDbhkYtAWP9jR-OiL3CpLiof0jVDDWEEN67PEeRCnvkr/file
Resolving uce1635c1f89abbc4e6f5ff98ac1.dl.dropboxusercontent.com (uce1635c1f89abbc4e6f5ff98ac1.dl.dropboxusercontent.com)... 162.125.8.15, 2620:100:

In [4]:
glass = pd.read_csv('data/glass.csv')
glass.columns = ['ri','na','mg','al','si','k','ca','ba','fe','glass_type']
glass.head(3)

,ri,na,mg,al,si,k,ca,ba,fe,glass_type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


As you know Train test validation splits are useful to get the best model, however if we have few datapoints sometimes is best to maximise the training data. For that we use **cross validation**. Scikit-learn has a very nice interface to do it:

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
glass['household'] =  glass.glass_type.apply(lambda x: int(x/4))
glass.head()

,ri,na,mg,al,si,k,ca,ba,fe,glass_type,household
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1,0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1,0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1,0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1,0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1,0


In [24]:
# hyperparameters of the model

from sklearn.linear_model import LogisticRegression

parameters = {'C': np.linspace(0.1, 10, 20)}
estimator = LogisticRegression()


In [25]:
feature_cols = ['al', 'na', 'fe', 'mg']
X = glass[feature_cols]
y = glass['household']


In [26]:
clf = GridSearchCV(estimator, parameters)
clf.fit(X,y)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([ 0.1       ,  0.62105263,  1.14210526,  1.66315789,  2.18421053,
        2.70526316,  3.22631579,  3.74736842,  4.26842105,  4.78947368,
        5.31052632,  5.83157895,  6.35263158,  6.87368421,  7.39473684,
        7.91578947,  8.43684211,  8.95789474,  9.47894737, 10.        ])})

In [29]:
clf

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([ 0.1       ,  0.62105263,  1.14210526,  1.66315789,  2.18421053,
        2.70526316,  3.22631579,  3.74736842,  4.26842105,  4.78947368,
        5.31052632,  5.83157895,  6.35263158,  6.87368421,  7.39473684,
        7.91578947,  8.43684211,  8.95789474,  9.47894737, 10.        ])})

In [30]:
clf.score(X, y)

0.9252336448598131